# Preparation


Check README.md file for install/setup instructions 

**References**


https://dvc.org/doc/tutorial/define-ml-pipeline - used as example

## Initialize DVC

In [1]:
!dvc --version #1.1.2

1.1.2


In [2]:
# Инициализация проекта DVC
!dvc init -f


You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: https://dvc.org/doc
- Get help and share ideas: https://dvc.org/chat
- Star us on GitHub: https://github.com/iterative/dvc


In [3]:
%%bash
# Первый коммит в Git 

git add .
git commit -m "Initialize DVC"

[master bf5eab4] Initialize DVC
 8 files changed, 3025 insertions(+), 715 deletions(-)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/default.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 tutorial_old.ipynb


### Files and Directories 

In [4]:
# Вывести список файлов и папок в подпапке `.dvc`, включая скрытые (флаг `-a`)
!ls -a .dvc 

.          ..         .gitignore config     plots      tmp


In [5]:
# Вывести содеержимое файла `.gitignore` в подпапке `.dvc`
!cat .dvc/.gitignore

/config.local
/tmp
/cache


# Control versions of data

In [6]:
# Get data (утилита wget скрее всего не сработает на Windows)
!wget -P data/ https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv
    
# Вывести использование места на диске файлами в папке `data`
!du -sh data/*

--2020-07-01 15:07:58--  https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.244.133
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 3716 (3,6K) [text/plain]
Сохранение в: «data/iris.csv»

iris.csv            100%[===================>]   3,63K  --.-KB/s    за 0s      

2020-07-01 15:07:58 (8,77 MB/s) - «data/iris.csv» сохранён [3716/3716]

4,0K	data/iris.csv


In [7]:
# Look on data

import pandas as pd

df = pd.read_csv('data/iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Add file under DVC control

In [8]:
%%bash
# добавить данные под контроль DVC
dvc add data/iris.csv
# Посмотреть размер файлов только в подпапке `data` (флаг -s) в килобайтах (флаг -h)
du -sh data/*


To track the changes with git, run:

	git add data/.gitignore data/iris.csv.dvc
4,0K	data/iris.csv
4,0K	data/iris.csv.dvc


In [9]:
!git status -s data/

?? data/.gitignore
?? data/iris.csv.dvc


In [10]:
%%bash

git add .
git commit -m "Add a source dataset"

[master d313c36] Add a source dataset
 2 files changed, 4 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/iris.csv.dvc


## What is DVC-file?

Data file internals


>    If you take a look at the DVC-file, you will see that only outputs are defined in outs. 
    In this file, only one output is defined. The output contains the data file path in the repository and md5 cache.
    This md5 cache determines a location of the actual content file in DVC cache directory .dvc/cache
    >> Output from DVC-files defines the relationship between the data file path in a repository and the path in a cache directory. See also DVC File Format



(c) dvc.org https://dvc.org/doc/tutorial/define-ml-pipeline

In [11]:
# Вывести содеержимое файла `iris.csv.dvc` в подпапке `data`
!cat data/iris.csv.dvc

outs:
- md5: 57fce90c81521889c736445f058c4838
  path: iris.csv


In [12]:
# Посмотреть размер файлов только в подпапке `.dvc/cache` (флаг -s) в килобайтах (флаг -h)
!du -sh .dvc/cache/*/*

4,0K	.dvc/cache/57/fce90c81521889c736445f058c4838


# Create ML pipeline

Stages 
- extract features 
- split dataset 
- train 
- evaluate 


DVC начиная с версии 1.0 хранит описания стадий пайплайнов в файле `dvc.yaml`

## Add feature extraction stage

In [13]:
# Создать стадию (пайплайн) для расчета фичей
!dvc run -n stage_feature_extraction \
    -d src/featurization.py \
    -d data/iris.csv \
    -o data/iris_featurized.csv \
    python src/featurization.py

Running stage 'stage_feature_extraction' with command:                          
	python src/featurization.py
Creating 'dvc.yaml'                                                             
Adding stage 'stage_feature_extraction' in 'dvc.yaml'
Generating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml data/.gitignore dvc.lock


In [14]:
# Посмотреть скрипт для фичей
# !cat src/featurization.py

In [15]:
# Посмотреть файлы в рабочей папке
!ls 

LICENSE            dvc.lock           src
README.md          dvc.yaml           tutorial.ipynb
data               requirements.txt   tutorial_old.ipynb


In [16]:
# Соержимое файла стадии
!cat dvc.yaml

stages:
  stage_feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv


In [17]:
import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [18]:
!git status -s

 M data/.gitignore
?? dvc.lock
?? dvc.yaml


In [19]:
%%bash
git add .
git commit -m "Add stage_features_extraction"

[master 12b885c] Add stage_features_extraction
 3 files changed, 19 insertions(+)
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml


## Add split train/test stage

In [20]:
# Создание стадии для разбивки датасета на трейн/тест с параметром `test_size`
!dvc run -n stage_split_dataset \
    -d src/split_dataset.py \
    -d data/iris_featurized.csv \
    -o data/train.csv \
    -o data/test.csv \
    python src/split_dataset.py --test_size 0.4

Running stage 'stage_split_dataset' with command:                               
	python src/split_dataset.py --test_size 0.4
Adding stage 'stage_split_dataset' in 'dvc.yaml'                                
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock data/.gitignore


In [21]:
!cat dvc.yaml

stages:
  stage_feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv
  stage_split_dataset:
    cmd: python src/split_dataset.py --test_size 0.4
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs:
    - data/test.csv
    - data/train.csv


## Add train stage

In [22]:
# Создание стадии для обучения модели
!dvc run -n stage_train \
    -d src/train.py \
    -d data/train.csv \
    -o data/model.joblib \
    python src/train.py

Running stage 'stage_train' with command:                                       
	python src/train.py
Adding stage 'stage_train' in 'dvc.yaml'                                        
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add data/.gitignore dvc.yaml dvc.lock


In [23]:
!cat dvc.yaml

stages:
  stage_feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv
  stage_split_dataset:
    cmd: python src/split_dataset.py --test_size 0.4
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs:
    - data/test.csv
    - data/train.csv
  stage_train:
    cmd: python src/train.py
    deps:
    - data/train.csv
    - src/train.py
    outs:
    - data/model.joblib


### Add evaluate stage

In [33]:
# Создание стадии для применения модели модели
!dvc run -f -n stage_evaluate \
    -d src/train.py \
    -d src/evaluate.py \
    -d data/test.csv \
    -d data/model.joblib \
    -m data/eval.json \
    python src/evaluate.py

Running stage 'stage_evaluate' with command:                                    
	python src/evaluate.py
Adding stage 'stage_evaluate' in 'dvc.yaml'                                     
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml data/.gitignore dvc.lock


In [34]:
!cat dvc.yaml

stages:
  stage_feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv
  stage_split_dataset:
    cmd: python src/split_dataset.py --test_size 0.4
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs:
    - data/test.csv
    - data/train.csv
  stage_train:
    cmd: python src/train.py
    deps:
    - data/train.csv
    - src/train.py
    outs:
    - data/model.joblib
  stage_evaluate:
    cmd: python src/evaluate.py
    deps:
    - data/model.joblib
    - data/test.csv
    - src/evaluate.py
    - src/train.py
    metrics:
    - data/eval.json


In [27]:
# !dvc dag 

# Metrics tracking

In [35]:
!dvc metrics show

	data/eval.json:                                                                
		f1_score: 0.7861833464670345


## Commit dvc pipelines

In [36]:
!git status -s

 M data/.gitignore
 M dvc.lock
 M dvc.yaml
 M src/evaluate.py
 M tutorial.ipynb
?? data/eval.txt


In [37]:
%%bash
git add .
git commit -m "Add pipelines"

[master 11bd63c] Add pipelines
 6 files changed, 136 insertions(+), 161 deletions(-)
 create mode 100644 data/eval.txt


# Reproducibility

## How does it work?

> The most exciting part of DVC is reproducibility.
>> Reproducibility is the time you are getting benefits out of DVC instead of spending time defining the ML pipelines.

> DVC tracks all the dependencies, which helps you iterate on ML models faster without thinking what was affected by your last change.
>> In order to track all the dependencies, DVC finds and reads ALL the DVC-files in a repository and builds a dependency graph (DAG) based on these files.

> This is one of the differences between DVC reproducibility and traditional Makefile-like build automation tools (Make, Maven, Ant, Rakefile etc). It was designed in such a way to localize specification of DAG nodes.
If you run repro on any created DVC-file from our repository, nothing happens because nothing was changed in the defined pipeline.

(c) dvc.org https://dvc.org/doc/tutorial/reproducibility

In [39]:
# Nothing to reproduce
!dvc repro stage_evaluate

Stage 'data/iris.csv.dvc' didn't change, skipping                               
Stage 'stage_feature_extraction' didn't change, skipping
Stage 'stage_split_dataset' didn't change, skipping
Stage 'stage_train' didn't change, skipping
Stage 'stage_evaluate' didn't change, skipping
Data and pipelines are up to date.


## Add features



### Create new experiment branch

Before editing the code/featurization.py file, please create and checkout a new branch __ratio_features__

In [40]:
# create new branch

!git checkout -b ratio_features
!git branch

Switched to a new branch 'ratio_features'
  master
* ratio_features


### Update featurization.py

in file __featurization.py__ uncomment lines 

    features['sepal_length_to_sepal_width'] = features['sepal_length'] / features['sepal_width']
    features['petal_length_to_petal_width'] = features['petal_length'] / features['petal_width']

In [41]:
!cat src/featurization.py

import pandas as pd


def get_features(dataset):

    features = dataset.copy()

    # uncomment for step 5.2  Add features
    features['sepal_length_to_sepal_width'] = features['sepal_length'] / features['sepal_width']
    features['petal_length_to_petal_width'] = features['petal_length'] / features['petal_width']

    return features


if __name__ == '__main__':

    dataset = pd.read_csv('data/iris.csv')

    features  = get_features(dataset)
    features.to_csv('data/iris_featurized.csv', index=False)


## Reproduce pipeline 

In [42]:
!dvc repro stage_evaluate

Stage 'data/iris.csv.dvc' didn't change, skipping                               
Running stage 'stage_feature_extraction' with command:
	python src/featurization.py
Updating lock file 'dvc.lock'                                                   

Running stage 'stage_split_dataset' with command:
	python src/split_dataset.py --test_size 0.4
Updating lock file 'dvc.lock'                                                   

Running stage 'stage_train' with command:
	python src/train.py
Updating lock file 'dvc.lock'                                                   

Running stage 'stage_evaluate' with command:
	python src/evaluate.py
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock


In [43]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length_to_sepal_width,petal_length_to_petal_width
0,5.1,3.5,1.4,0.2,setosa,1.457143,7.0
1,4.9,3.0,1.4,0.2,setosa,1.633333,7.0
2,4.7,3.2,1.3,0.2,setosa,1.468750,6.5
3,4.6,3.1,1.5,0.2,setosa,1.483871,7.5
4,5.0,3.6,1.4,0.2,setosa,1.388889,7.0


## Compare metrics for all runs (experiments)

In [44]:
# this pipeline metrics 

!dvc metrics show

	data/eval.json:                                                                
		f1_score: 0.8084886128364389


In [45]:
# show all commited pipelines metrics 

!dvc metrics show -a

workspace:                                                                      
	data/eval.json:
		f1_score: 0.8084886128364389
master, ratio_features:
	data/eval.json:
		f1_score: 0.7861833464670345


In [47]:
!dvc metrics diff 

Path            Metric    Value    Change                                       
data/eval.json  f1_score  0.80849  0.02231


## Commit new results

In [48]:
!git status -s

 M dvc.lock
 M src/featurization.py
 M tutorial.ipynb


In [49]:
!git add .
!git commit -m "New features experiment"

[ratio_features cc24d56] New features experiment
 3 files changed, 77 insertions(+), 88 deletions(-)


# Checkout (start over new experiment)

- in case new features doesn't result improvements 
- or we want to improve the model by changing the hyperparameters (with OLD dataset)

## Checkout code and data files 

In [52]:
%%bash
# Переход обратно на исходную ветку
git checkout master
dvc checkout

Your branch is ahead of 'origin/master' by 4 commits.
  (use "git push" to publish your local commits)
M	data/eval.json
M	data/test.csv
M	data/train.csv
M	data/iris_featurized.csv
M	data/model.joblib


Switched to branch 'master'


In [53]:
!git branch

* master
  ratio_features


In [54]:
!dvc metrics show

	data/eval.json:                                                                
		f1_score: 0.7861833464670345


In [55]:
# Nothing to reproduce since code was checked out by `git checkout`
# and data files were checked out by `dvc checkout`
!dvc repro stage_evaluate

Stage 'data/iris.csv.dvc' didn't change, skipping                               
Stage 'stage_feature_extraction' didn't change, skipping
Stage 'stage_split_dataset' didn't change, skipping
Stage 'stage_train' didn't change, skipping
Stage 'stage_evaluate' didn't change, skipping
Data and pipelines are up to date.


In [56]:
!dvc metrics show

	data/eval.json:                                                                
		f1_score: 0.7861833464670345


In [57]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Tune the model

In [58]:
# create new branch for experiment

!git checkout -b tuning
!git branch

Switched to a new branch 'tuning'
  master
  ratio_features
* tuning


### Change parameters of classifier (LogisticRegression)

in file __train.py__ in constructor of LogisticRegression:

* change C param to 0.1

in the end you should get:

```python
clf = LogisticRegression(C=0.1, solver='newton-cg', multi_class='multinomial', max_iter=100)
```

### Reproduce pipelines

In [59]:
# re-run pipeline 

!dvc repro stage_evaluate

Stage 'data/iris.csv.dvc' didn't change, skipping                               
Stage 'stage_feature_extraction' didn't change, skipping
Stage 'stage_split_dataset' didn't change, skipping
Running stage 'stage_train' with command:
	python src/train.py
Updating lock file 'dvc.lock'                                                   

Running stage 'stage_evaluate' with command:
	python src/evaluate.py
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock


In [60]:
!cat data/eval.json

{"f1_score": 0.9639376218323586, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 17, 0], [0, 2, 18]]}}

In [61]:
!dvc metrics show -a

workspace:                                                                      
	data/eval.json:
		f1_score: 0.9639376218323586
master, tuning:
	data/eval.json:
		f1_score: 0.7861833464670345
ratio_features:
	data/eval.json:
		f1_score: 0.8084886128364389


In [63]:
!dvc metrics diff

Path            Metric    Value    Change                                       
data/eval.json  f1_score  0.96394  0.17775


In [64]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### Commit

In [65]:
%%bash

git add .
git commit -m "Tune model. C=0.1"

[tuning fdba0ed] Tune model. C=0.1
 3 files changed, 300 insertions(+), 109 deletions(-)


### Merge the model to dvc-tutorial

In [66]:
%%bash

git checkout master
git merge tuning

Your branch is ahead of 'origin/master' by 4 commits.
  (use "git push" to publish your local commits)
Updating 11bd63c..fdba0ed
Fast-forward
 dvc.lock       |  10 +-
 src/train.py   |   2 +-
 tutorial.ipynb | 397 ++++++++++++++++++++++++++++++++++++++++++---------------
 3 files changed, 300 insertions(+), 109 deletions(-)


Switched to branch 'master'


# Share data

## Setup remote storage (i.e. cloud)

In [67]:
# Create new remote

!dvc remote add -d local /tmp/dvc

Setting 'local' as a default remote.


In [68]:
# as you can see, .dvc/config is changed

!git status -s

 M .dvc/config
 M tutorial.ipynb


In [69]:
# check config file 

!cat .dvc/config

[core]
    remote = local
['remote "local"']
    url = /tmp/dvc


In [70]:
%%bash

git add .
git commit -m "Add remote storage"

[master 6e6c573] Add remote storage
 2 files changed, 178 insertions(+), 12 deletions(-)


## Push data to remote

In [71]:
# Push data to remote

!dvc push

6 files pushed                                                                  


## Pull date from remote

In [72]:
!dvc pull

Everything is up to date.                                                       
